### C. Hydrogen solubility and diffusivity in bulk α-iron

In [20]:
import pyiron
import ase.calculators
import ase.optimize
import ase.neb
import ase.visualize
import ase.build
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize

In [2]:
potential =  'FeH-NNIP'

1. solution energy

$$E_s = E_{Fe+H} - E_{Fe}- E_{H_2}/2$$

$E_{H_2}/2$: the energy of a gaseous $H_2$ molecule

In [16]:
# calculation of E_Fe
pr = pyiron.Project('solution_E')
job = pr.create.job.Lammps('bulk', delete_existing_job=True)
bulk = pr.create.structure.bulk('Fe', cubic=True).repeat([4,4,4])
job.structure = bulk
job.potential = potential
job.calc_minimize(pressure=0.0)
job.run(delete_existing_job=True)

E_Fe = job.output.energy_tot[-1]

The job bulk was saved and received the ID: 10913


In [11]:
# H at O-site
cell = bulk.get_cell()
bulk_H_O = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.87/2, 2.87, 2.87/2)], cell=cell)
bulk_H_O += H
bulk_H_O.plot3d()

job = pr.create.job.Lammps('bulk_H_O', delete_existing_job=True)
job.structure = bulk_H_O
job.potential = potential
job.calc_minimize(pressure=0.0)
job.run(delete_existing_job=True)

E_Fe_H_O = job.output.energy_tot[-1]
E_Fe_H_O

The job bulk_H_O was saved and received the ID: 10911


-1057.90317567852

In [14]:
# H at T-site

bulk_H_T = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.87/4*6, 2.87/4*8, 2.87/4*6-2.87/4)], cell=cell)
bulk_H_T += H
bulk_H_T.plot3d()

job = pr.create.job.Lammps('bulk_H_T', delete_existing_job=True)
job.structure = bulk_H_T
job.potential = potential
job.calc_minimize(pressure=0.0)
job.run(delete_existing_job=True)

E_Fe_H_T = job.output.energy_tot[-1]
E_Fe_H_T

The job bulk_H_T was saved and received the ID: 10912


-1058.06512033958

In [15]:
E_Fe_H_T - E_Fe_H_O

-0.16194466106003347

-0.15300000000000002 in paper

### Old version

In [3]:
# calculation of E_Fe
pr = pyiron.Project('solution_E')
job = pr.create.job.Lammps('bulk', delete_existing_job=True)
bulk = pr.create.structure.bulk('Fe', cubic=True).repeat([4,4,4])
job.structure = bulk
job.potential = potential
job.calc_minimize(pressure=0.0)
job.run(delete_existing_job=True)

The job bulk was saved and received the ID: 10908


KeyboardInterrupt: 

In [23]:
E_Fe = job.output.energy_tot[-1]
bulk = job.get_structure(-1)
cell = bulk.get_cell()

In [37]:
# H at O-site
bulk_H_O = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/2, 2.83, 2.83/2)], cell=cell)
bulk_H_O += H
bulk_H_O.plot3d()

NGLWidget()

In [38]:
job = pr.create.job.Lammps('bulk_H_O', delete_existing_job=True)
job.structure = bulk_H_O
job.potential = potential
job.calc_minimize()
job.run(delete_existing_job=True)

The job bulk_H_O was saved and received the ID: 10595


In [39]:
E_Fe_H_O = job.output.energy_tot[-1]
E_Fe_H_O

-448.064760912784

In [40]:
E_Fe_H_O - E_Fe

-3.021180453103966

In [41]:
# H at T-site

bulk_H_T = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4*6-2.83/4)], cell=cell)
bulk_H_T += H
bulk_H_T.plot3d()

NGLWidget()

In [42]:
job = pr.create.job.Lammps('bulk_H_T', delete_existing_job=True)
job.structure = bulk_H_T
job.potential = potential
job.calc_minimize()
job.run(delete_existing_job=True)

The job bulk_H_T was saved and received the ID: 10596


In [43]:
E_Fe_H_T = job.output.energy_tot[-1]
E_Fe_H_T

-448.175938372699

In [51]:
E_Fe_H_T -E_Fe_H_O

-0.1111774599150408

In [46]:
0.236 - 0.389
# E difference on paper

-0.15300000000000002

In [105]:
job = pr.load('bulk_H_T')
job.animate_structures()

NGLWidget(max_frame=1)

In [106]:
job.output.energy_tot

array([-447.9359933 , -448.17593837])

? \
$E_{Fe+H}$ = ...\
after inserting H, $E_{Fe+H}$ should be the energy before or after relaxation\
$E_{H_2}$ ~ 4.5 eV?

In [107]:
job = pr.load('bulk_H_O')
E_H_O_ms = job.output.energy_tot[0]
job = pr.load('bulk_H_T')
E_H_T_ms = job.output.energy_tot[-1]

E_H_T_ms - E_H_O_ms

-0.75238956341002

In [101]:
# print(E_Fe_H_T -  E_Fe)
# print(E_Fe_H_O -  E_Fe)
# E_Fe_H_T - E_Fe_H_O

2. Diffusion energy barrier (T to T)

In [60]:
bulk_H_T_imageI = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4*6-2.83/4)], cell=cell)
bulk_H_T_imageI += H
bulk_H_T_imageI += H
bulk_H_T_imageI.plot3d()

NGLWidget()

In [61]:
bulk_H_T_imageF = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4)], cell=cell)
bulk_H_T_imageF += H
bulk_H_T_imageF.plot3d()

NGLWidget()

In [63]:
IS = bulk_H_T_imageI.to_ase()
FS = bulk_H_T_imageF.to_ase()

LAMMPS calculator:

In [2]:
from ase import Atom, Atoms
from ase.build import bulk
from ase.calculators.lammpsrun import LAMMPS

parameters = {'pair_style': 'eam/alloy',
            'pair_coeff': ['* * NiAlH_jea.eam.alloy H Al']}

files = ['NiAlH_jea.eam.alloy']

Ni = bulk('Ni', cubic=True)
H = Atom('H', position=Ni.cell.diagonal()/2)
NiH = Ni + H

lammps = LAMMPS(parameters=parameters, files=files)

NiH.calc = lammps
NiH.get_potential_energy()

RuntimeError: Failed to retrieve any thermo_style-output

https://wiki.fysik.dtu.dk/ase/ase/calculators/lammpsrun.html#lammps

In [1]:
from lammps import lammps
lmp = lammps()

LAMMPS (7 Feb 2024 - Update 1)


In [5]:
from ase import Atom, Atoms
from ase.build import bulk
from ase.calculators.lammpslib import LAMMPSlib

In [3]:
cmds = ["pair_style eam/alloy",
        "pair_coeff * * NiAlH_jea.eam.alloy Ni H"]

In [4]:
Ni = bulk('Ni', cubic=True)
H = Atom('H', position=Ni.cell.diagonal()/2)
NiH = Ni + H

In [5]:
lammps = LAMMPSlib(lmpcmds=cmds, log_file='test.log')

In [6]:
NiH.calc = lammps

In [7]:
lammps

In [8]:
NiH.get_total_energy()

-19.919477009060433

3. Diffusivity of H atoms (see diffusivity and C)

In [48]:
job.list_potentials()

['1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1',
 '2011--Ko-W-S--Ni-H--LAMMPS--ipr1',
 '2013--Shim-J-H--V-Ni-H--LAMMPS--ipr1',
 'EAM_Dynamo_AngeloMoodyBaskes_1995_NiAlH__MO_418978237058_005',
 'EAM_Dynamo_TehranchiCurtin_2010_NiH__MO_535504325462_003',
 'MEAM_LAMMPS_KoShimLee_2011_NiH__MO_091278480940_000',
 'MEAM_LAMMPS_ShimKoKim_2013_NiVH__MO_612225165948_000']

In [49]:
job.potential = job.list_potentials()[0]

In [50]:
job.calc_minimize()

In [51]:
job.run(delete_existing_job=True)

The job NiH was saved and received the ID: 10813


In [52]:
job.output.energy_tot

array([-19.91947701, -20.02129346, -20.06472795, -20.06739502])

#### set up lammps calculator with nnip

In [53]:
from ase.calculators.lammpslib import LAMMPSlib
import ase.visualize
from ase import Atom, Atoms
from ase.build import bulk
import pyiron
from ase.optimize import BFGS

In [2]:
# cmds = ["pair_style eam/alloy",
#         "pair_coeff * * NiAlH_jea.eam.alloy Ni H"]

In [51]:
cmds = ['pair_style hdnnp 6.5 dir  ./  showew no showewsum 0 resetew yes maxew 1000000 cflength 1.8897261328 cfenergy 0.0367493254\n', 
 'pair_coeff * * Fe H\n']

In [54]:
Fe = bulk('Fe', cubic=True).repeat([2,2,2])
del Fe[14]
H = Atom('H', position=(2.87,2.87,2.87))
FeH = Fe + H

In [55]:
ase.visualize.view(FeH)

<Popen: returncode: None args: ['/home/gu.huang/miniconda3/envs/pyiron/bin/p...>

In [56]:
lammps = LAMMPSlib(lmpcmds=cmds, log_file='test.log')

In [57]:
FeH.calc = lammps

In [58]:
FeH.get_total_energy()

-123.97297094105927

In [59]:
opt = BFGS(FeH)
opt.run(fmax = 0.01)

      Step     Time          Energy          fmax
BFGS:    0 13:17:34     -123.972971        0.495822
BFGS:    1 13:17:34     -123.999201        0.430154
BFGS:    2 13:17:34     -124.081780        0.018775
BFGS:    3 13:17:34     -124.081942        0.000398


True

In [60]:
Fe.calc = lammps
Fe.get_total_energy()

-121.17179825573427

In [61]:
opt = BFGS(Fe)
opt.run(fmax = 0.01)

      Step     Time          Energy          fmax
BFGS:    0 13:18:07     -121.171798        0.284121
BFGS:    1 13:18:07     -121.180355        0.243018
BFGS:    2 13:18:07     -121.204354        0.009609


True

In [62]:
FeH.get_total_energy()

-124.08194223517538

In [11]:
Fe.calc = lammps

In [27]:
Fe = bulk('Fe', cubic=True)
Fe.calc = lammps
Fe.get_total_energy()

-16.460429871064395

In [4]:
ase.visualize.view(Fe)

<Popen: returncode: None args: ['/home/gu.huang/miniconda3/envs/pyiron/bin/p...>

In [11]:
pr = pyiron.Project('FeH_ase')

In [13]:
job = pr.create.job.Lammps('FeH_ase', delete_existing_job=True)
job.structure = pyiron.ase_to_pyiron(FeH)
job.potential = job.list_potentials()[0]

In [15]:
job.calc_minimize()

In [16]:
job.run()

The job FeH_ase was saved and received the ID: 10814


In [17]:
job.output.energy_tot

array([-17.03694527, -18.18694344])

In [22]:
from ase.optimize import BFGS

In [23]:
opt = BFGS(FeH)

In [ ]:
opt.run(fmax=0.01)

      Step     Time          Energy         fmax
BFGS:    0 11:02:00      -17.036945        7.6779


In [31]:
import ase.constraints
import ase.optimize 
ucell = ase.constraints.StrainFilter(Fe, mask=[1, 1, 1, 0, 0, 0])
opt = ase.optimize.QuasiNewton(ucell, logfile="Fe_optimize-cell.log", trajectory="Fe_optimize-cell.traj")
opt.run(fmax=0.01)

/tmp/ipykernel_2271270/1287121399.py:3: FutureWarning: Import StrainFilter from ase.filters
  ucell = ase.constraints.StrainFilter(Fe, mask=[1, 1, 1, 0, 0, 0])


True

In [32]:
Fe.get_cell()

Cell([2.8299844434291943, 2.8299844434291925, 2.829984443429211])

In [34]:
import ase.io
import ase.visualize
atoms = ase.io.read("./Fe_optimize-cell.traj")
ase.visualize.view(atoms)

<Popen: returncode: None args: ['/home/gu.huang/miniconda3/envs/pyiron/bin/p...>

2. Diffusion energy barrier (T to T)

In [63]:
from ase.calculators.lammpslib import LAMMPSlib
from ase.visualize import view
from ase import Atom, Atoms
import ase.build
import pyiron
from ase.optimize import BFGS
import pyiron

In [49]:
bulk = ase.build.bulk('Fe', cubic=True).repeat([3,3,3])
bulk.calc = 

In [35]:
bulk_H_T_imageI = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4*6-2.83/4)], cell=cell)
bulk_H_T_imageI += H
bulk_H_T_imageI += H
bulk_H_T_imageI.plot3d()

AttributeError: 'function' object has no attribute 'copy'

In [61]:
bulk_H_T_imageF = bulk.copy()
H = pr.create.structure.atoms('H', positions=[(2.83/4*6, 2.83/4*8, 2.83/4)], cell=cell)
bulk_H_T_imageF += H
bulk_H_T_imageF.plot3d()

NGLWidget()

In [63]:
IS = bulk_H_T_imageI.to_ase()
FS = bulk_H_T_imageF.to_ase()